In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## F-MNIST

In [2]:
dl_train = torch.utils.data.DataLoader(
    torchvision.datasets.FashionMNIST('./data/f_mnist', train=True, download=True))

dl_test  = torch.utils.data.DataLoader(
    torchvision.datasets.FashionMNIST('./data/f_mnist', train=False, download=True))

Using downloaded and verified file: ./data/f_mnist/FashionMNIST/raw/train-images-idx3-ubyte.gz
Extracting ./data/f_mnist/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/f_mnist/FashionMNIST/raw


EOFError: Compressed file ended before the end-of-stream marker was reached

In [ ]:
train_data   = dl_train.dataset.data.to(dtype=torch.float32)
train_labels = dl_train.dataset.targets

In [ ]:
fig_mnist, ax = plt.subplots(1,8, figsize=(8*4,4))
for i in range(8):
    ax[i].imshow(train_data[i].numpy(), cmap='Greys');

|Label|Description|
|-----|-----------|
| 0  |T-shirt/top| 
| 1  | Trouser   |
| 2  | Pullover  |
| 3  | Dress     |
| 4  | Coat      |
| 5  | Sandal   |
| 6  | Shirt    |
|7 | Sneaker|
|8| Bag |
|9| Ankle boot|

In [ ]:
train_labels[0:8]

In [ ]:
train_dataset = torch.utils.data.TensorDataset( 
    (train_data).view(-1,28*28), 
    train_labels)

In [ ]:
train_dataset, validation_dataset = torch.utils.data.random_split(train_dataset,(50000,10000))

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=100, 
                                           shuffle=True)

In [ ]:
test_data   = dl_test.dataset.data.to(dtype=torch.float32)
test_labels = dl_test.dataset.targets
test_dataset = torch.utils.data.TensorDataset(
    (test_data/256).view(-1,28*28), test_labels)

In [ ]:
from torch import nn

In [ ]:
model = torch.nn.Sequential(
    nn.Linear(28*28,1024), nn.ReLU(),
    nn.Linear(1024,512), nn.ReLU(),
    nn.Linear(512,256), nn.ReLU(),
    nn.Linear(256,128), nn.ReLU(),
    nn.Linear(128,10)
)

In [ ]:
optim = torch.optim.Adam(model.parameters(),lr=0.0001)

In [ ]:
loss_f = nn.CrossEntropyLoss()

In [ ]:
errors = []
batches = 0
epochs = 0

In [ ]:
%%time
for e in range(5):
    for d in train_loader:        
        optim.zero_grad()
        features, labels = d
        pred = model(features)
        loss = loss_f(pred, labels)
        errors.append(loss.item())
        loss.backward()
        optim.step()
        batches += 1
    epochs += 1   
print(loss)        

In [ ]:
plt.plot(np.linspace(0,epochs, batches),errors)

In [ ]:
with torch.no_grad():
    pred = torch.softmax(model(train_dataset[:][0]),1)
    train_ac = torch.sum(torch.argmax(pred,1)==train_dataset[:][1]).to(dtype=torch.float32)/len(train_dataset)
train_ac  

In [ ]:
with torch.no_grad():
    pred = torch.softmax(model(validation_dataset[:][0]),1)
    validation_ac = torch.sum(torch.argmax(pred,1)==validation_dataset[:][1]).to(dtype=torch.float32)/len(validation_dataset)
validation_ac    

In [ ]:
#torch.save(model,"f_mnist.pt")